# AI-Frameworks

<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 
<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" width=400, style="max-width: 150px; display: inline"  alt="Wikistat"/></a>
<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" width=400,  style="float:right;  display: inline" alt="IMT"/> </a>
</center>

# LAB 5 Introduction to Recommendation System with Collaborative Filtering  -  Part 2 : Latent Vector-Based Methods with `nmf` and `softImpute` R Library.

The objectives of this notebook are the following : 

* Discover `nmf` and `softImpute`  R library.
* Discover Latent-Based methods to apply recommendation system.
    * Latent-Based methods enable to decompose the X rating matrices into sub matrices that enable to represen to represent user and item in a smaller space and enable to represent interaction between them.

* Use different factorization algorithms to learn decomposition of rating's matrices.
* [Singular value decomposition](http://wikistat.fr/pdf/st-m-explo-alglin.pdf) (**SVD**) and [*Non Negativ Matrix Factorization*](http://wikistat.fr/pdf/st-m-explo-nmf.pdf) (**NMF**).
* Use softImpute to performe matrices rating's completion.
* Use results of algorithm to apply recommendation. 


**NB** : When data are rates, "0" usually mean that the value is not here. In this case, NMF or SVD are not suitable. However most of the implementation allow to use those algorithms and consider "0" as a "0" rate.  This is the case of `scikit-learn` and `surprise` NMF and SVD implementation.
In this situation we should prefer completion algorithm.

# Toy Dataset

To understand well how factorization algorithm works, we will used  a toy dataset called 'recom-jouet.dat'. 
It contains information on how much products have been bought (or notation on the product) by a user. You can change its value if you want.

In [ ]:
jouet=read.table("recom-jouet.dat")
jouet

In [ ]:
options(repr.plot.width=6, repr.plot.height=3)
boxplot(jouet)

Data are sparse. The variable have very different variances.
We might want to normalize the data as user can have different scale of notation.

## Recommandation par NMF
The R package `NMF` contains various NMF algorithms

In [ ]:
#install.packages("NMF")
library("NMF")
nmfAlgorithm()

### Select the best algorithm

We will look for the best NMF algorithm among these four algorithms. 

In [ ]:
nmfAlgorithm("brunet")

In [ ]:
nmfAlgorithm("lee")

In [ ]:
nmfAlgorithm("snmf/l")

In [ ]:
nmfAlgorithm("snmf/r")

**Q** What are the loss function associated to those four algorithm? Do they use regularization?

We now compare the four methods on toy dataset, using  rank 5. 
We perform 10 runs for each of them as results depends of the initialization. 

In [ ]:
res.multi.method=nmf(jouet, 5,nrun=10, list("brunet","lee","snmf/l","snmf/r"), 
                     seed = 111, .options ="t")
compare(res.multi.method)

The `NMF` package proposes a lot of tools to visualize results and help you understand them. 
The *consensusmap* allow to display, for each method, different information such that:

* the heatmap of the consensus matrix,
* the *basis*  that show for each column, the dominant basis component in the best fit
* The *consensus* which is the result of the consensus matrix.


The **consensus matrix** is the ratio between the sum of connectivity matrix of all run and the number of run. 

THe **connectivity matrix** is a binary matrix build over the result of one run of nmf algorithm. The entry (i,j) is equal to one if column i and j belong to the same cluster. 0 otherwise.  An entry i and j belong to the same cluster if they have the same dominant basis. 

In [ ]:
options(repr.plot.width=6, repr.plot.height=6)
consensusmap(res.multi.method,hclustfun="ward") 

**Q** What represent those different plot? 
**Q** Which method is the best?

### Select the rank

We now perform the NMF algorithm with the best methods. And we run this method according to various rank (from 2 to 6). 

In [ ]:
estim.r=nmf(jouet,2:6,method="snmf/l", nrun=10,seed=111)

We can now look at various indicators to look at the performance of the different run and choose the best rank.

In [ ]:
plot(estim.r)  

**Q** what do these different metrics represent? 

In [ ]:
options(repr.plot.width=8, repr.plot.height=5)
consensusmap(estim.r) 

**Q** According to the different metric, which rank seems the better?

### Explore results 
Now we have choosen a method and a rank, let's iterate several time to get the best run.

In [ ]:
nmf.jouet=nmf(jouet,4,method="snmf/l",nrun=30,seed=111)

We can now extract the **W** and **H** matrix in order to visualize the data and explore it.

In [ ]:
w=basis(nmf.jouet)
h=coef(nmf.jouet)

It's now possible to construct the `basis matrix`. 
In this matrix, each entry (i,j) represent the ratio of the number of time a user i had j as a dominant basis component over the number of run. 

The `basismap` enables to display the heatmap of the basis matrix as well as the result of a hierarchical clustering on this matrix. 

In [ ]:
options(repr.plot.width=4, repr.plot.height=3)
basismap(nmf.jouet,hclustfun="ward")

The `coefmap` is the equivalent of `basismap` for the items 

In [ ]:
options(repr.plot.width=5, repr.plot.height=3)
coefmap(nmf.jouet,hclustfun="ward")

We perform now hierarchical clustering directly on the embedding representation the users.
The distance use to compute the clustering is the euclidean distances.

In [ ]:
distmod.h=dist(scale(t(h)), method="eucl")
hclusmod.h=hclust(distmod.h,method="ward.D")
options(repr.plot.width=5, repr.plot.height=4)
plot(hclusmod.h)

We apply now a MDS on the distance matrix in order to represent the user in a 2D plot. It could be done on PCA to.

In [ ]:
dN.h=dimnames(h)[[2]]
hclasmod.h = cutree(hclusmod.h,k=4)
mdjouet= cmdscale(distmod.h, k=2)
plot(mdjouet, type="n", xlab="", ylab="",main="")
text(mdjouet,dN.h,col=hclasmod.h)
abline(v=0,h=0)

We perform now hierarchical clustering directly on the embedding representation the items.

In [ ]:
distmod.v=dist(scale(w), method="eucl")
mdjouet= cmdscale(distmod.v, k=2)
hclusmod.v=hclust(distmod.v,method="ward.D")
plot(hclusmod.v)

In [ ]:
hclasmod.v = cutree(hclusmod.v,k=2)
dN.v=dimnames(w)[[1]]
plot(mdjouet, type="n", xlab="", ylab="",main="")
text(mdjouet,dN.v,col=hclasmod.v)
abline(v=0,h=0)

We now want to display information on both user and items on the same graph. This can be done with the `aheatmap` function.

In [ ]:
# intégration des deux classifications
aheatmap(jouet,Rowv=hclusmod.v, Colv=hclusmod.h,annRow=as.factor(hclasmod.v),
         annCol=as.factor(hclasmod.h))

We can also display both user items on their same graph according to axis of their embeddings.

In [ ]:
plot(t(h)[,2:3]/max(h), type="n", xlab="", ylab="",main="", xlim=c(-0.2,1.2))
text(t(h)[,2:3]/max(h),dN.h)
abline(v=0,h=0)
points(w[,2:3]/max(w), type="n", xlab="", ylab="",main="")
text(w[,2:3]/max(w),dN.v, col="red")
abline(v=0,h=0)

### Recommandation

The NMF package does not contains method to directy perform recommendation.
However this can easily be done by computing the estimation 
$\hat{x}=w*h$ of the $x$ origin matrix.


In [ ]:
# Matrice reconstruite
xchap=w%*%h

We can know apply the recommendation by detecting the higher reconstruct score of the matrix.

In [ ]:
prod=apply(xchap-10*jouet,1,function(x) which.max(x))
cbind(dN.v,dN.h[prod])

## Par SVD

SVD decomposition generate a factorization **X**=**UL V'** which has better properties than the one produces by the NMF as the unique solution for a given rank.


#### Recommandation
Approximation de rang 2 par SVD

In [ ]:
res=svd(jouet)
# Matrice reconstruite
xchap=res$u[,1:2]%*%diag(res$d[1:2])%*%t(res$v[,1:2])

In [ ]:
prod=apply(xchap-10*jouet,1,function(x) which.max(x))
cbind(dN.v,dN.h[prod])

**Q** Compare this recommendation with the one perform by NMF. 

## Matrix Completion 


We now consider "0" as a missing value. The problem is know a completion matrix problem.
The `SoftImpute` library enables to apply softumpute algorithm (see lectures slide.), which can be compared to a thresholded SVD.

We first replace 0 by "NA" value

In [ ]:
jouet.na=jouet
jouet.na[jouet==0]=NA

The recommendation can then be applied the same way than with the SVD.

In [ ]:
install.packages("softImpute")
library(softImpute)
res=softImpute(jouet.na,rank.max=2,type="svd",lambda=1)
# Matrice reconstruite
xchap=res$u%*%diag(res$d)%*%t(res$v)

In [ ]:
prod=apply(xchap-10*jouet,1,function(x) which.max(x))
cbind(dN.v,dN.h[prod])

**Q** Compare the results.

This study is only a introduction to collaborative filtering with latent vectors-based methods.

A lot of questions hasn't been discussed such has *cold start* problem.

# Movielens

See the notebook `1-Python-Neighborhood-MovieLens.ipynb` for an introduction to these data.

We start by studying the small dataset composed f 100k rows.  
We download the updated ratings filed build in the previous notebook in order to compare results on same test/train dataset.

In [ ]:
ratings=read.csv("movielens_small/ratings_updated.csv")
head(ratings, 5)

We download the test_id produced in first notebook in order to reproduce the same train test partition.

In [ ]:
test=ratings[ratings$test_train=="test",1:3]
train=ratings[ratings$test_train=="train",1:3]

## NMF

In order to perform NMF algorithms on the movielens dataset, we need to build a dense matrix from the ratings data, as the `nmf` package does not take sparse matrix into account. 
This is a strong limitation that does not allow you to perform factorization on the dataset in a reasonable amount of time for this lab (even in the small dataset!).
We won't perform factorizaton on this dataset. 

You can easily perform this factorization with **python** with either `scikit-learn` or `surprise` library. 
Surprise provide a benchmark of the performance of various algorithm on this dataset (http://surpriselib.com/#benchmarks). 
However their are only on implementation of the NMF and it doesn't not handle missing data. This is why we won't cover this implementation in this course.

## SoftImpute

On the contrary, `SoftImpute`package allow to easily perform  *softImpute* algorithm on sparse data

In [ ]:
#install.packages("softImpute")
library(softImpute)

In [ ]:
# Mise au format d'une matrice creuse
trainSparse=Incomplete(train$userId,train$movieId,train$rating)
# appel de la fonction
res=softImpute(trainSparse,rank.max=4,type="als",lambda=1,maxit=200)
# complétion
recom=impute(res,test[,1],test[,2])

In [ ]:
# Calcul de l'erreur (RMSE)
sqrt(mean((test[,3]-recom)**2))

The error is quite bad compare to the ones obtained with neighborhood methods.  However, the advantage of this method is that it can be used in a reasonable amount of time on bigger dataset.

### Complete dataset

In [ ]:
# Lecture de la matrice
dBrut=read.csv("ml-25/ratings.csv")

In [ ]:
# Extraction de l'échantillon test
dTestInd=sample(nrow(dBrut),nrow(dBrut)/10,replace=FALSE)
dTest=dBrut[dTestInd,1:3]
nrow(dTest)

In [ ]:
# sous-échantillonage de l'échantillon d'apprentissage
dInter=dBrut[-dTestInd,1:3]
taux=0.1
dTrainInd=sample(nrow(dInter),nrow(dInter)*taux,replace=FALSE)
dTrain=dInter[dTrainInd,1:3]
# Matrice d'échantillonnage sparse
dTrainSparse=Incomplete(dTrain$userId,dTrain$movieId,dTrain$rating)

In [ ]:
# Factorisation
t1=Sys.time()
res=softImpute(dTrainSparse,rank.max=10,type="als",lambda=20,maxit=200)
t2=Sys.time()
# Reconstruction
recom=impute(res,dTest[,1],dTest[,2])
#RMSE
sqrt(mean((dTest[,3]-recom)**2))
difftime(t2,t1) 

Apprentissage avec fichier complet (taux=1): 20M de notes

rang | lambda | temps (') | rmse 
----|---------|-------|-----
 4  |  1       |  5.6 |  1.07   
 10 |  10  |  12.6 |  1.02  
 10 |  20  |  12.2 |  1.033 
 15 |  10  |  19.4 |  1.016
 20 |   1  |  26.9  | 1.02 
 20 |  10  |  26.1  | 1.016 
 20 |  15  |  24.4 |  1.018
 20 |  20  |  27.0  | 1.016 
 30 |  20  |  40.1 |  1.02
